# Training and Inferencing Model Using Data Cleaning Methods

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA

In [3]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge

pd.set_option('max_colwidth',200)
pd.set_option('display.width',200)
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',1000)

#from xgboost import XGBRegressor

# 1. Data Cleaning

In [4]:
full = pd.read_excel('datasets/new_data_1.xls')
full = full.drop(columns=["Qingpu","0"])
full = full.loc[:, ~full.columns.str.contains('^Unnamed')]
full.head()

,Price,Area,Bedrooms,Livingrooms,Bathrooms,Floor,NextToSubway,Exquisite,OpenForVisits,Longitude,Latitude,863098207ffffff,86309820fffffff,863098217ffffff,86309821fffffff,863098227ffffff,86309822fffffff,863098237ffffff,863098247ffffff,86309824fffffff,863098257ffffff,86309825fffffff,863098267ffffff,86309826fffffff,863098277ffffff,863098287ffffff,86309828fffffff,863098297ffffff,86309829fffffff,8630982afffffff,8630982c7ffffff,8630982cfffffff,8630982d7ffffff,8630982dfffffff,8630982e7ffffff,8630982efffffff,8630982f7ffffff,863098307ffffff,863098347ffffff,86309834fffffff,863098357ffffff,86309835fffffff,86309836fffffff,86309838fffffff,8630983afffffff,86309864fffffff,863098657ffffff,863098667ffffff,86309874fffffff,863098967ffffff,863098c57ffffff,863099407ffffff,86309940fffffff,863099417ffffff,86309941fffffff,863099427ffffff,863099437ffffff,863099487ffffff,86309948fffffff,863099497ffffff,86309949fffffff,8630994a7ffffff,8630994afffffff,8630994b7ffffff,8630994c7ffffff,8630994cfffffff,8630994d7ffffff,8630994dfffffff,8630994e7ffffff,8630994efffffff,8630994f7ffffff,863099507ffffff,863099517ffffff,86309951fffffff,863099527ffffff,863099537ffffff,863099587ffffff,86309958fffffff,863099597ffffff,86309959fffffff,8630995a7ffffff,8630995afffffff,8630995b7ffffff,863099c27ffffff,863099c87ffffff,863099c97ffffff,863099c9fffffff,86309aa67ffffff,86309b117ffffff,86309b147ffffff,86309b167ffffff,86309b177ffffff,86309b807ffffff,86309b80fffffff,86309b827ffffff,86309b82fffffff,86309b837ffffff,86309b847ffffff,86309b84fffffff,86309b857ffffff,86309b85fffffff,86309b867ffffff,86309b86fffffff,86309b877ffffff,86309b89fffffff,86309b8a7ffffff,86309b8afffffff,86309b8c7ffffff,86309b8cfffffff,86309b8efffffff,86309b907ffffff,86309b90fffffff,86309b917ffffff,86309b91fffffff,86309b927ffffff,86309b92fffffff,86309b937ffffff,86309b947ffffff,86309b94fffffff,86309b957ffffff,86309b95fffffff,86309b967ffffff,86309b96fffffff,86309b977ffffff,86309b99fffffff,86309b9afffffff,86309ba0fffffff,86309ba17ffffff,86309ba27ffffff,86309ba2fffffff,86309ba57ffffff,86309ba8fffffff,86309bacfffffff,86309badfffffff,86309baf7ffffff,86309bb07ffffff,86309bb0fffffff,86309bb17ffffff,86309bb1fffffff,86309bb27ffffff,86309bb37ffffff,86309bb57ffffff,86309bb77ffffff,86309bb8fffffff,86309bb9fffffff,86309bba7ffffff,86309bbb7ffffff,86309bc4fffffff,86309bc67ffffff,86309bd6fffffff,86309d68fffffff,86318c74fffffff,86401970fffffff,864118b27ffffff,864189057ffffff
0,6700,65.00,2.0,0,NaN,NaN,1,NaN,NaN,121.489484,31.240373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8500,91.00,2.0,2,NaN,NaN,1,NaN,NaN,121.460855,31.295211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2600,56.07,2.0,1,NaN,NaN,1,NaN,NaN,121.440338,31.419139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5500,84.00,2.0,2,NaN,NaN,1,NaN,NaN,121.460855,31.295211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6800,78.00,1.0,2,NaN,NaN,1,NaN,NaN,121.543922,31.278916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [5]:
for col in full:
    full[col] = pd.to_numeric(full[col], errors='coerce')
full.head()

,Price,Area,Bedrooms,Livingrooms,Bathrooms,Floor,NextToSubway,Exquisite,OpenForVisits,Longitude,Latitude,863098207ffffff,86309820fffffff,863098217ffffff,86309821fffffff,863098227ffffff,86309822fffffff,863098237ffffff,863098247ffffff,86309824fffffff,863098257ffffff,86309825fffffff,863098267ffffff,86309826fffffff,863098277ffffff,863098287ffffff,86309828fffffff,863098297ffffff,86309829fffffff,8630982afffffff,8630982c7ffffff,8630982cfffffff,8630982d7ffffff,8630982dfffffff,8630982e7ffffff,8630982efffffff,8630982f7ffffff,863098307ffffff,863098347ffffff,86309834fffffff,863098357ffffff,86309835fffffff,86309836fffffff,86309838fffffff,8630983afffffff,86309864fffffff,863098657ffffff,863098667ffffff,86309874fffffff,863098967ffffff,863098c57ffffff,863099407ffffff,86309940fffffff,863099417ffffff,86309941fffffff,863099427ffffff,863099437ffffff,863099487ffffff,86309948fffffff,863099497ffffff,86309949fffffff,8630994a7ffffff,8630994afffffff,8630994b7ffffff,8630994c7ffffff,8630994cfffffff,8630994d7ffffff,8630994dfffffff,8630994e7ffffff,8630994efffffff,8630994f7ffffff,863099507ffffff,863099517ffffff,86309951fffffff,863099527ffffff,863099537ffffff,863099587ffffff,86309958fffffff,863099597ffffff,86309959fffffff,8630995a7ffffff,8630995afffffff,8630995b7ffffff,863099c27ffffff,863099c87ffffff,863099c97ffffff,863099c9fffffff,86309aa67ffffff,86309b117ffffff,86309b147ffffff,86309b167ffffff,86309b177ffffff,86309b807ffffff,86309b80fffffff,86309b827ffffff,86309b82fffffff,86309b837ffffff,86309b847ffffff,86309b84fffffff,86309b857ffffff,86309b85fffffff,86309b867ffffff,86309b86fffffff,86309b877ffffff,86309b89fffffff,86309b8a7ffffff,86309b8afffffff,86309b8c7ffffff,86309b8cfffffff,86309b8efffffff,86309b907ffffff,86309b90fffffff,86309b917ffffff,86309b91fffffff,86309b927ffffff,86309b92fffffff,86309b937ffffff,86309b947ffffff,86309b94fffffff,86309b957ffffff,86309b95fffffff,86309b967ffffff,86309b96fffffff,86309b977ffffff,86309b99fffffff,86309b9afffffff,86309ba0fffffff,86309ba17ffffff,86309ba27ffffff,86309ba2fffffff,86309ba57ffffff,86309ba8fffffff,86309bacfffffff,86309badfffffff,86309baf7ffffff,86309bb07ffffff,86309bb0fffffff,86309bb17ffffff,86309bb1fffffff,86309bb27ffffff,86309bb37ffffff,86309bb57ffffff,86309bb77ffffff,86309bb8fffffff,86309bb9fffffff,86309bba7ffffff,86309bbb7ffffff,86309bc4fffffff,86309bc67ffffff,86309bd6fffffff,86309d68fffffff,86318c74fffffff,86401970fffffff,864118b27ffffff,864189057ffffff
0,6700,65.00,2.0,0,NaN,NaN,1,NaN,NaN,121.489484,31.240373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8500,91.00,2.0,2,NaN,NaN,1,NaN,NaN,121.460855,31.295211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2600,56.07,2.0,1,NaN,NaN,1,NaN,NaN,121.440338,31.419139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5500,84.00,2.0,2,NaN,NaN,1,NaN,NaN,121.460855,31.295211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6800,78.00,1.0,2,NaN,NaN,1,NaN,NaN,121.543922,31.278916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [6]:
aa = full.isnull().sum()
aa[aa>0].sort_values(ascending=False)

Bathrooms        19031
OpenForVisits     7496
Exquisite         7496
Floor             3018
Latitude          1477
Longitude         1477
Bedrooms             1
dtype: int64

In [7]:
print("NAME     count    mean    std    min    25%    50%    75%    max")
for column in full.columns:
    print(column,
          full[column].count(), 
          full[column].mean(),
          full[column].std(),
          full[column].min(),
          full[column].quantile(0.25),
          full[column].quantile(0.5),
          full[column].quantile(0.75),
          full[column].max()
         )
    

NAME     count    mean    std    min    25%    50%    75%    max
Price 19415 11354.14854493948 11821.337036964045 499 4300.0 6800.0 16000.0 400000
Area 19415 105.36554880247161 82.6100116750667 6.0 51.0 75.0 150.0 3218.0
Bedrooms 19414 2.3567528587617184 1.422442111226409 0.0 1.0 2.0 3.0 9.0
Livingrooms 19415 1.437136234869946 0.631453996192551 0 1.0 1.0 2.0 6
Bathrooms 384 1.7473958333333333 1.0478093470490075 0.0 1.0 1.0 2.0 7.0
Floor 16397 16.30761724705739 11.140945304221189 1.0 6.0 17.0 27.0 121.0
NextToSubway 19415 0.5620911666237445 0.496142485328652 0 0.0 1.0 1.0 1
Exquisite 11919 0.6939340548703751 0.46087677666552834 0.0 0.0 1.0 1.0 1.0
OpenForVisits 11919 0.19011662052185585 0.3924094932967419 0.0 0.0 0.0 0.0 1.0
Longitude 17938 121.45541007503495 0.18855527977951422 106.510455 121.37935875 121.453308 121.53699925000001 121.925622
Latitude 17938 31.21349035829091 0.12214522201606778 23.134378 31.178389000000006 31.2193 31.238452 39.015151
863098207ffffff 19415 0.002214782384

In [7]:
print(full['Longitude'].mean())
print(full['Latitude'].mean())

121.45541007503495
31.21349035829091


## Cliping

In [8]:

full = full[full['Price']<=40000]
full = full[full['Price']>1000]

full.shape

(19137, 155)

In [9]:
train, validate= np.split(full.sample(frac=1, random_state=42), [int(.8*len(full))])

### Filling Empty Datas

In [10]:
def fill_empty(input_set, output_set):
    # fill with 0
    cols=["Exquisite", "OpenForVisits"]
    for col in cols:
        output_set[col].fillna(0, inplace=True)
        
    # fill with functions
    temp = []
    areas = output_set["Area"]
    for a in areas:
        t = math.floor(a/120)+1
        temp.append(t)
    output_set["temp"] = temp
    output_set["Bathrooms"].fillna(output_set["temp"], inplace=True)
    output_set = output_set.drop(columns=["temp"])
    
    # fill with input mean value
    cols=["Bedrooms","Floor", "Longitude","Latitude"]
    for col in cols:
        output_set[col].fillna(input_set[col].mean(), inplace=True)
    output_set["Floor"]=output_set["Floor"].astype(np.int)
    output_set["Bedrooms"]=output_set["Bedrooms"].astype(np.int)
    
    return output_set
    
    

In [11]:
train = fill_empty(train, train)
validate = fill_empty(train, validate)

In [12]:
#train.head()
validate.head()

,Price,Area,Bedrooms,Livingrooms,Bathrooms,Floor,NextToSubway,Exquisite,OpenForVisits,Longitude,Latitude,863098207ffffff,86309820fffffff,863098217ffffff,86309821fffffff,863098227ffffff,86309822fffffff,863098237ffffff,863098247ffffff,86309824fffffff,863098257ffffff,86309825fffffff,863098267ffffff,86309826fffffff,863098277ffffff,863098287ffffff,86309828fffffff,863098297ffffff,86309829fffffff,8630982afffffff,8630982c7ffffff,8630982cfffffff,8630982d7ffffff,8630982dfffffff,8630982e7ffffff,8630982efffffff,8630982f7ffffff,863098307ffffff,863098347ffffff,86309834fffffff,863098357ffffff,86309835fffffff,86309836fffffff,86309838fffffff,8630983afffffff,86309864fffffff,863098657ffffff,863098667ffffff,86309874fffffff,863098967ffffff,863098c57ffffff,863099407ffffff,86309940fffffff,863099417ffffff,86309941fffffff,863099427ffffff,863099437ffffff,863099487ffffff,86309948fffffff,863099497ffffff,86309949fffffff,8630994a7ffffff,8630994afffffff,8630994b7ffffff,8630994c7ffffff,8630994cfffffff,8630994d7ffffff,8630994dfffffff,8630994e7ffffff,8630994efffffff,8630994f7ffffff,863099507ffffff,863099517ffffff,86309951fffffff,863099527ffffff,863099537ffffff,863099587ffffff,86309958fffffff,863099597ffffff,86309959fffffff,8630995a7ffffff,8630995afffffff,8630995b7ffffff,863099c27ffffff,863099c87ffffff,863099c97ffffff,863099c9fffffff,86309aa67ffffff,86309b117ffffff,86309b147ffffff,86309b167ffffff,86309b177ffffff,86309b807ffffff,86309b80fffffff,86309b827ffffff,86309b82fffffff,86309b837ffffff,86309b847ffffff,86309b84fffffff,86309b857ffffff,86309b85fffffff,86309b867ffffff,86309b86fffffff,86309b877ffffff,86309b89fffffff,86309b8a7ffffff,86309b8afffffff,86309b8c7ffffff,86309b8cfffffff,86309b8efffffff,86309b907ffffff,86309b90fffffff,86309b917ffffff,86309b91fffffff,86309b927ffffff,86309b92fffffff,86309b937ffffff,86309b947ffffff,86309b94fffffff,86309b957ffffff,86309b95fffffff,86309b967ffffff,86309b96fffffff,86309b977ffffff,86309b99fffffff,86309b9afffffff,86309ba0fffffff,86309ba17ffffff,86309ba27ffffff,86309ba2fffffff,86309ba57ffffff,86309ba8fffffff,86309bacfffffff,86309badfffffff,86309baf7ffffff,86309bb07ffffff,86309bb0fffffff,86309bb17ffffff,86309bb1fffffff,86309bb27ffffff,86309bb37ffffff,86309bb57ffffff,86309bb77ffffff,86309bb8fffffff,86309bb9fffffff,86309bba7ffffff,86309bbb7ffffff,86309bc4fffffff,86309bc67ffffff,86309bd6fffffff,86309d68fffffff,86318c74fffffff,86401970fffffff,864118b27ffffff,864189057ffffff
16340,1800,55.0,1,1,1.0,9,0,0.0,0.0,121.352695,31.408544,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1227,4600,105.0,3,2,1.0,16,1,0.0,0.0,121.424385,31.044272,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1387,3600,38.0,1,1,1.0,16,1,0.0,0.0,121.566194,31.260599,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3994,16000,137.0,3,2,2.0,33,1,1.0,0.0,121.475281,31.206211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11448,30000,208.0,4,2,2.0,26,1,1.0,0.0,121.574193,31.234138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Saving Train & Test

In [13]:
train.to_excel('train.xls')
validate.to_excel('validate.xls')